In [1]:
using Turing
using DataFrames
using CSV
using Random
using Distributions
using StatisticalRethinking
using StatisticalRethinking: link
using StatisticalRethinkingPlots
using StatsPlots
using StatsBase
using Logging

default(label=false);
Logging.disable_logging(Logging.Warn);

# 14.1 Varying slopes by construction